# Pin array creation 

In [4]:
import pandas as pd
import os, shutil
from tqdm import tqdm
import cv2

## Helper function

def createDirectory(path):
    if not os.path.isdir(path):
        os.makedirs(path)
        print ("Creating {}".format(path))
    return path

### Read 
- Read combined files from ZL dataset

### Body extraction
- This code extracts only the body and writes to a file


### Bugfix
-  Created to fix the mistake of 1 is pin

### Extracting the pin+pina array information only
- No body
- Merges the pin_array with original dataframe

In [92]:
def save_pins_only(name_list_dir, df):
    src_img_dir="D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_01/train"
    
    dst_dir=createDirirectory("D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_02/temp")
    
    name_list=os.listdir(name_list_dir)
    pbar=tqdm(name_list)
    for name in pbar:
        _df=df[df.filename==name.replace(".txt", ".jpg")]
        src_txt=os.path.join(name_list_dir, name)
        with open(os.path.join(dst_dir, name.replace(".jpg", ".txt")), "w") as f:
            for i, row in _df.iterrows():
                if (row["class"].upper()=="PIN"):
                    lbl=2
                    f.writelines("{},{},{},{},{}\n".format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], lbl))
            
            with open(src_txt, "r") as f_src:
                lines = f_src.readlines()
                for l in lines: f.writelines(l)
        #break


------------------------------------------------------------------------

### Pin array analysis

In [4]:
%clear all

In [1]:
def bb_intersection_ratio(boxA, boxB):
    xA = max(float(boxA["x1"]), float( boxB["x1"]))
    yA = max(float(boxA["y1"]), float(boxB["y1"]))
    xB = min(float(boxA["x2"]), float(boxB["x2"]))
    yB = min(float(boxA["y2"]), float(boxB["y2"]))
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (float(boxA["x2"]) - float(boxA["x1"]) + 1) * (float(boxA["y2"]) - float(boxA["y1"]) + 1)
    percentage_area = interArea / float(boxAArea ) *100
    return percentage_area

def group_pin_by_pinarray(A, B):
    for i, rowB in B.iterrows():
        B.at[i,'group']=i
        for j, rowA in A.iterrows():
            percentage_area = bb_intersection_ratio(rowA, rowB)
            if (percentage_area >30):
                A.at[j,'group']=i
                # print(rowA)
    return A, B

def load_label_df(txt_fpath, img_fpath):
    extended=pd.read_csv(txt_fpath, names=['x1', 'y1', 'x2', 'y2','class'])
    extended["group"]=-1
    extended[extended["class"]==2], extended[extended["class"]==0] = group_pin_by_pinarray(extended[extended["class"]==2], extended[extended["class"]==0])
    return extended

In [18]:
# def multiline(img, windowX):
#     height, width, channels = img.shape
#     split = ceil(width / windowX)
#     for s in range(split):
#         crop_x1 = s * windowX
#         crop_x2
#         crop_img = img[:, crop_x1:crop_x2]
    
def save_seperate_groups(txt_fpath, img_fpath, single_component_df):
    group_list = list(single_component_df.groupby("group").count().index)
    group_list.remove(-1)
#     print (group_list)
#     print(single_component_df[single_component_df['class']==0])
#     print(txt_fpath,"\n", img_fpath)
    
    bname=os.path.basename(txt_fpath)[:-4]
    for i, gid in enumerate(group_list):
        #Paths
        out_dir=createDirectory(os.path.dirname(txt_fpath.replace("txt_pin_pinarray","txt_pin_pinarray_sep___")))
        out_txt_path = os.path.join(out_dir, bname+"___A{}.txt".format(i))
        out_img_path = out_txt_path.replace(".txt", ".jpg")
        
        #DF
        selected_component_df = single_component_df[single_component_df["group"]==gid]
        del selected_component_df['group']
        selected_pin_df = selected_component_df[selected_component_df['class']==2]
        selected_pinarray_df = selected_component_df[selected_component_df['class']==0]
        selected_pin_df['x1'].iloc[:] = selected_pin_df['x1'].iloc[:]- int(selected_pinarray_df.loc[:,'x1'])
        selected_pin_df['x2'].iloc[:] = selected_pin_df['x2'].iloc[:]- int(selected_pinarray_df.loc[:,'x1'])
        selected_pin_df['y1'].iloc[:] = selected_pin_df['y1'].iloc[:]- int(selected_pinarray_df.loc[:,'y1'])
        selected_pin_df['y2'].iloc[:] = selected_pin_df['y2'].iloc[:]- int(selected_pinarray_df.loc[:,'y1'])
        
        #Write
        selected_pin_df.to_csv(out_txt_path, header=False, index=False)
        shutil.copy(img_fpath, out_dir)
        shutil.move(os.path.join(out_dir, os.path.basename(img_fpath)), out_txt_path.replace(".txt", ".jpg"))
        
        #Image write
        img =cv2.imread(img_fpath, 0)
        # print("img.shape: ", img.shape)
        # print("selected_pinarray_df: ", selected_pinarray_df)
        
        crop_x1, crop_x2, crop_y1, crop_y2 = int(selected_pinarray_df["x1"]), int(selected_pinarray_df["x2"]), int(selected_pinarray_df["y1"]), int(selected_pinarray_df["y2"])
        crop_img = img[crop_y1:crop_y2, crop_x1:crop_x2]
        # print(crop_img.shape, crop_x1, crop_x2, crop_y1,crop_y2)
        cv2.imwrite(out_img_path, crop_img)
        
# save_seperate_groups(txt_fpath,img_fpath, extended)

---------------------------------------
### Save individual pin slice.

In [7]:
import glob
filename='D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_02/txt_pin_pinarray'
filename='D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray'

names = glob.glob(os.path.join(filename, '*.jpg'))
for img_name in names:
    img_fpath=img_name
    print(img_name)
    txt_fpath=img_name.replace(".jpg", ".txt")
    extend_df = load_label_df(txt_fpath, img_fpath)
    save_seperate_groups(txt_fpath,img_fpath, extend_df)
    
    

D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray\4609080-H-TOP-NEW-0626_22_0_1-u6.jpg


C:\Users\V510\Anaconda3\envs\TF_15_Py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\V510\Anaconda3\envs\TF_15_Py36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\V510\Anaconda3\envs\TF_15_Py36\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\V510\Anaconda3\envs\TF_15_Py36\lib\site-packag

D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray\4609080-H-TOP-NEW-0626_22_1_1-u6.jpg
D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray\4609080-H-TOP-NEW-0626_22_2_1-u6.jpg
D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray\4609080-H-TOP-NEW-0626_22_4_1-u6.jpg
D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray\4609080-H-TOP-NEW-0626_22_5_1-u6.jpg
D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-ZL/txt_pin_pinarray\4609080-H-TOP-NEW-0626_22_8_1-u6.jpg
D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/images/H_Dataset_08/H_Dataset_02_PinArray/QFP-Grayscale-Z